In [2]:
#Add needed imports
import numpy as np
import pandas as pd
import os
import shap
from numpy import mean
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import RobustScaler
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier  
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow import keras
import numpy as np
import keras 
import pydot
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from keras.utils import plot_model
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
import os
import shap

#Read data
proccessed_data_path =os.path.join(os.path.pardir,os.path.pardir,'data','processed')
train_path = os.path.join(proccessed_data_path,'dataset3.csv')
df = pd.read_csv(train_path)
labels=df['Churn']
x = df.drop(columns=['Churn','Unnamed: 0'],axis = 'columns')
y=np.ravel(labels)
oversample = SMOTENC(categorical_features=[ 2,3,4,5,18])
x, y = oversample.fit_resample(x, y)
sc = RobustScaler()
x = pd.DataFrame(sc.fit_transform(x),columns = x.columns)

ii = 1

xgb_model = XGBClassifier(random_state=1,learning_rate=0.05, max_depth=7,eval_metric='mlogloss',use_label_encoder =False ,objective="binary:logistic")
dt_model=DecisionTreeClassifier(random_state=1,criterion='entropy',max_depth = 7,min_samples_leaf=30)  
nn_model = Sequential()
nn_model.add(Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.001), input_dim=19, activation='relu' ))
nn_model.add(Dropout(rate=0.2))
nn_model.add(Dense(8,kernel_regularizer=tf.keras.regularizers.l2(0.001),activation='relu'))
nn_model.add(Dropout(rate=0.1))
nn_model.add(Dense(1, activation='sigmoid'))

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=70,restore_best_weights=True)
cv = StratifiedKFold(n_splits=10)
for train_index, test_index in cv.split(x, y):
	print("Iteration" , ii)
	Xi_train, Xi_test = x.loc[train_index], x.loc[train_index]
	yi_train, yi_test = y[train_index], y[test_index]
	xgb_model.fit(Xi_train,yi_train)
	shap_values = shap.TreeExplainer(xgb_model).shap_values(Xi_test,approximate=True)
	#shap.summary_plot(shap_values, Xi_test, plot_type="bar")
	vals = np.abs(shap_values).mean(axis=0)
	print(pd.DataFrame(list(zip(vals)),columns=['XGB - feature_importance_vals']).to_string(index=False))    
	Xi_train, Xi_test = x.loc[train_index], x.loc[test_index]
	yi_train, yi_test = y[train_index], y[test_index]
	dt_model.fit(Xi_train,yi_train)
	explainer = shap.TreeExplainer(dt_model)
	shap_values = explainer.shap_values(Xi_test)
	#shap.summary_plot(shap_values[1], Xi_test, plot_type="bar")
	vals = np.abs(shap_values[1]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['DT - feature_importance_vals']).to_string(index=False))
	lr_schedule= tf.keras.optimizers.schedules.InverseTimeDecay( 0.001,decay_steps=(Xi_train.shape[0]/32)*50,decay_rate=1,staircase=False)
	nn_model.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(lr_schedule), metrics=['accuracy'])
	nn_model.fit(Xi_train, yi_train, validation_data=(Xi_test, yi_test), epochs=150, batch_size=10,verbose=0,callbacks=[callback])
	explainer = shap.DeepExplainer(nn_model, Xi_train[1:100].to_numpy())
	shap_values = explainer.shap_values(Xi_test[1:20].to_numpy())
	vals = np.abs(shap_values[0]).mean(axis=0)
	print( pd.DataFrame(list(zip(vals)),columns=['NN - feature_importance_vals']).to_string(index=False))
	ii += 1                           

Iteration 1


ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.164765
                      0.035180
                      0.006683
                      0.029090
                      0.013873
                      0.000253
                      0.039054
                      0.586989
                      0.106617
                      0.057937
                      0.085188
                      0.717108
                      0.055933
                      0.234924
                      1.346942
                      2.451812
                      0.319421
                      0.031038
                      0.071711
 DT - feature_importance_vals
                     0.027455
                     0.000387
                     0.000000
                     0.000000
                     0.000362
                     0.000000
                     0.000523
                     0.096756
                     0.001092
                     0.000160
                     0.008582
                    

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.131448
                      0.032164
                      0.013855
                      0.025737
                      0.022101
                      0.006478
                      0.028841
                      0.466256
                      0.074507
                      0.081976
                      0.074837
                      0.732874
                      0.076642
                      0.234569
                      1.324458
                      2.520676
                      0.439081
                      0.030973
                      0.058856
 DT - feature_importance_vals
                     0.018083
                     0.000000
                     0.000000
                     0.000215
                     0.000000
                     0.000000
                     0.000000
                     0.076006
                     0.000000
                     0.000000
                     0.000000
                    

...

 XGB - feature_importance_vals
                      0.167967
                      0.041175
                      0.012065
                      0.024049
                      0.012060
                      0.004706
                      0.033489
                      0.542769
                      0.118651
                      0.061573
                      0.082456
                      0.877771
                      0.068326
                      0.302528
                      1.373451
                      2.071790
                      0.439664
                      0.024962
                      0.054218
 DT - feature_importance_vals
                     0.006429
                     0.000000
                     0.000000
                     0.000254
                     0.000000
                     0.000000
                     0.000915
                     0.064242
                     0.000000
                     0.000103
                     0.000000
                    

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.171913
                      0.028392
                      0.014195
                      0.021068
                      0.009264
                      0.005479
                      0.022726
                      0.641142
                      0.102685
                      0.046773
                      0.097361
                      0.783370
                      0.057788
                      0.255156
                      1.464511
                      2.135344
                      0.367086
                      0.037349
                      0.055068
 DT - feature_importance_vals
                     0.006747
                     0.000000
                     0.000000
                     0.000000
                     0.000000
                     0.000000
                     0.001007
                     0.087029
                     0.000642
                     0.000000
                     0.000000
                    

ntree_limit is deprecated, use `iteration_range` or model slicing instead.


 XGB - feature_importance_vals
                      0.165972
                      0.038649
                      0.009055
                      0.016624
                      0.009077
                      0.004658
                      0.032640
                      0.540890
                      0.116865
                      0.059646
                      0.071417
                      0.848001
                      0.067919
                      0.287005
                      1.506371
                      1.974649
                      0.435514
                      0.027831
                      0.059275
 DT - feature_importance_vals
                     0.009741
                     0.000826
                     0.000000
                     0.000297
                     0.000000
                     0.000000
                     0.000941
                     0.069579
                     0.002317
                     0.000000
                     0.000000
                    